In [ ]:
import sys
import os

spark_home='/opt/spark-1.5.1-bin-hadoop2.6/'
# addin pyspark to current path
sys.path.append( spark_home+'/python' )
sys.path.append( spark_home+'/python/lib/py4j-0.8.2.1-src.zip' )

# Установка локальных переменных
os.environ["SPARK_HOME"] = spark_home
os.environ["HADOOP_HOME"] = '/opt/cloudera/parcels/CDH/lib/hadoop'
os.environ["HADOOP_YARN_HOME"] = '/opt/cloudera/parcels/CDH/lib/hadoop-yarn'
os.environ["YARN_CONF_DIR"] = '/etc/hadoop/conf.cloudera.yarn'
os.environ["SPARK_CLASSPATH"] = '/etc/hive/conf.cloudera.hive1'
os.environ["PYSPARK_SUBMIT_ARGS"] = '--master local[4] pyspark-shell'

from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext
conf = SparkConf ().set( 'spark.app.name', 'test');
sc = SparkContext (conf= conf)

Загрузка датасета, с которым мы проводим эсперименты

In [ ]:
# загружаем датасет
from pyspark.mllib.util import MLUtils
examples = MLUtils.loadLibSVMFile(sc, "/testdata/mllib_demo/libsvm_dataset")


# извлекаем фичи из датасета
features = examples.map (lambda x: x.features)

Метод визуализации ROC кривой

In [ ]:
def plotROC (real_labels, predicted_labels):
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, _ = roc_curve(real_labels, predicted_labels)
    roc_auc = auc (fpr, tpr)

    %matplotlib inline
    import matplotlib.pyplot as plt
    plt.clf()
    plt.plot(fpr,tpr)
    print "ROC AUC =", roc_auc

# <br><br><br><br>Кластеризация K средних

Обучение модели KMeans.train с параметрами
- **dataset** - датасет векторов
- **k** - число кластеров
- **maxIterations** - число итераций сходимости
- **initializationMode** - дотупные варианты "k-means||" (по умолчанию), "random"
- **runs** - (по умолчанию 1)

In [ ]:
from pyspark.mllib.clustering import KMeans

# Build the model (cluster the data)
clusters = KMeans.train(features, 2, maxIterations=10,
        runs=10, initializationMode="random")

In [ ]:
plotROC (examples.map (lambda x: x.label).collect(),
         examples.map (lambda x: 1-clusters.predict (x.features)).collect())